# Instalando a aplicação simul-shop

O um simul-shop é um simulador de loja online.

Inspirado no micro-sock, desenvolvida pela [weaveworks](https://www.weave.works/) e [distribuída](https://github.com/microservices-demo/microservices-demo/tree/master) de forma gratuíta (licença [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0)) para demonstrar seu produto [Waeve Scope](https://www.weave.works/oss/scope/).

O simulador foi construída usando [Python](https://www.python.org/) e é empacotado uma imagem Docker e entregue em um cluster de kubernetes.

## Simul Shop implementação

O simulador não tem uma interface web é apenas um programa em python que recebe requisições e invoca outras URLs com base nos parâmetros passados por variável de ambiente.

Por exemplo, vamos simular uma aplicação que tem um front-end e um backend, usando apenas o docker podemos executar da seguinte forma:

![](media/simul-shop-fb.png)

```bash
# Create net
docker network create my-net

# backend
docker run -d --rm \
--hostname backend \
--network my-net \
--name backend \
-e SCHED_CALL_URL_LST=http://localhost:8000/healthz \
-e SCHED_CALL_INTERVAL=5 \
kdop/generic-service:0.0.1

# front-end
docker run -d --rm \
    --network my-net \
    --hostname front-end \
    --name front-end \
    -e SCHED_CALL_URL_LST=http://front-end:8000/s \
    -e SCHED_CALL_INTERVAL=10 \
    -e SPLIT_CALL_URL_LST=http://backend:8000 \
    kdop/generic-service:0.0.1
```

Vamos acompanhar os _logs_ para ver o que está acontecento.

```bash
# Terminal 1
docker logs -f front-end
# Terminal 2
docker logs -f backend
```

Para remover os recursos

```bash
# Parando e excluindo os containers
docker kill front-end backend
# Clean-up
docker network rm my-net
```

### Istio + Simul Shop

Configuramos o simul-shop para criar uma malha de serviços, sua arquittura é:

![arquitetura](media/simul-shop-arq.png)

Iremos utiliza-lo para explorar os recursos do Istio.

## Instalando a aplicação

Com o clone do repo istio-curso, você tem um diretório com a aplicação.

In [2]:
ls -la exemplos/simul-shop-manifests

total 24
drwxr-xr-x   4 anderson  staff   128 Oct 22 14:34 .
drwxr-xr-x  10 anderson  staff   320 Oct 22 14:36 ..
-rw-r--r--   1 anderson  staff  5607 Oct 22 14:35 deployments.yaml
-rw-r--r--   1 anderson  staff  2638 Oct 22 14:34 services.yaml


A configuração para kubernetes compreende apenas dois recursos, o _deployment_ e o _service_ para cada um dos módulos da nossa aplicação, de acordo coma arquitetura da seção anterior.

* [exemplos/simul-shop-manifests/deployments.yaml](exemplos/simul-shop-manifests/deployments.yaml)
* [exemplos/simul-shop-manifests/services.yaml](exemplos/simul-shop-manifests/services.yaml)

Como na aplicação de demo, criamos um deployment para a imagem `kdop/generic-service 0.0.1` e adiconamos os _labels_ para representar o nome do módulo e sua versão.

```yaml
labels:
    app: "nome-do-modulo"
    version: "versao"
```

Os serviços foram configurados para acessar cada um dos módulos:

```yaml
metadata:
  name: front-end # <--- Cria um DNS com nome nome-modulo.
...
spec:
  type: ClusterIP
  selector:
    app: front-end # <--- E aponta para um POD que tennha o label nome-modulo.
```

Dessa forma os serviços podem ser endereçados na forma http://nome-modulo:porta/. Na nossa arquitetura:

* http://front-end:80000
* http://login:80000
* http://catalogue:80000
* http://orders:80000
* http://shipping:80000
* http://cart:80000
* http://payment:80000
* http://accounts:80000
* http://orders-db:80000
* http://queue:80000
* http://cart-db:80000
* http://accounts-db:80000


